#4-1

In [1]:
import keras
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

In [2]:
model = ResNet50(weights='imagenet')

In [6]:
img_path = 'pic1.jpg'
img_path2 = 'pic2.png'
img_path3 = 'pic3.jpg'
images = [img_path,img_path2,img_path3]

In [7]:

for image in images:
    img = keras.utils.load_img(image, target_size=(224, 224))
    x = keras.utils.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)

    print('Predicted:', decode_predictions(preds, top=3)[0])

1/1 [==============================] - 0s 65ms/step
Predicted: [('n03720891', 'maraca', 0.70883465), ('n03627232', 'knot', 0.12974116), ('n06596364', 'comic_book', 0.09969014)]
1/1 [==============================] - 0s 65ms/step
Predicted: [('n09472597', 'volcano', 0.5052898), ('n09468604', 'valley', 0.24283153), ('n03891251', 'park_bench', 0.058718342)]
1/1 [==============================] - 0s 67ms/step
Predicted: [('n02124075', 'Egyptian_cat', 0.5128608), ('n02123045', 'tabby', 0.16944222), ('n04589890', 'window_screen', 0.14576758)]


In [8]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

In [9]:
model = VGG16(weights='imagenet')

In [10]:

for image in images:
    img = keras.utils.load_img(image, target_size=(224, 224))
    x = keras.utils.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)

    print('Predicted:', decode_predictions(preds, top=3)[0])

1/1 [==============================] - 0s 192ms/step
Predicted: [('n03720891', 'maraca', 0.7146983), ('n06596364', 'comic_book', 0.11047901), ('n03724870', 'mask', 0.043612972)]
1/1 [==============================] - 0s 95ms/step
Predicted: [('n09468604', 'valley', 0.3789382), ('n09472597', 'volcano', 0.15442286), ('n09193705', 'alp', 0.039117113)]
1/1 [==============================] - 0s 96ms/step
Predicted: [('n02124075', 'Egyptian_cat', 0.3933159), ('n02123045', 'tabby', 0.34667197), ('n02123159', 'tiger_cat', 0.1569642)]


In [14]:
#EfficientNetV2M
from keras.applications.efficientnet import EfficientNetB7
from keras.applications.efficientnet import preprocess_input

In [15]:
model = EfficientNetB7(weights='imagenet')

In [16]:
for image in images:
    img = keras.utils.load_img(image, target_size=(600, 600))
    x = keras.utils.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)

    print('Predicted:', decode_predictions(preds, top=3)[0])

1/1 [==============================] - 4s 4s/step
Predicted: [('n03888257', 'parachute', 0.32841113), ('n03627232', 'knot', 0.12577166), ('n02782093', 'balloon', 0.088213265)]
1/1 [==============================] - 1s 537ms/step
Predicted: [('n09468604', 'valley', 0.32421666), ('n02859443', 'boathouse', 0.16733956), ('n02793495', 'barn', 0.13650851)]
1/1 [==============================] - 1s 534ms/step
Predicted: [('n02123045', 'tabby', 0.5062912), ('n02123159', 'tiger_cat', 0.2739568), ('n02124075', 'Egyptian_cat', 0.076381475)]


#4-2

In [21]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow_datasets.public_api as tfds
import tensorflow as tf
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [18]:
batch_size = 32
img_height = 64
img_width = 64
num_classes = 10
epochs = 10


In [19]:
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'eurosat',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
    batch_size=batch_size
)

In [31]:
class_labels = metadata.features['label'].names
print(class_labels)

['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']


In [14]:

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

#classification layers
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

#transfer learning model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_ds, epochs=epochs, batch_size=batch_size, validation_data=val_ds)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_ds)
print("Test Accuracy:", test_accuracy)

# Fine-tuning (Optional)
# Uncomment the following lines to unfreeze some or all of the pre-trained layers for fine-tuning
# for layer in base_model.layers:
#     layer.trainable = True

# model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
# model.fit(train_data, train_labels, epochs=num_epochs, batch_size=batch_size, validation_data=(val_data, val_labels))


Epoch 1/10
675/675 [==============================] - 88s 129ms/step - loss: 0.9772 - accuracy: 0.8311 - val_loss: 0.4467 - val_accuracy: 0.8674
Epoch 2/10
675/675 [==============================] - 84s 124ms/step - loss: 0.2779 - accuracy: 0.9119 - val_loss: 0.4297 - val_accuracy: 0.8819
Epoch 3/10
675/675 [==============================] - 84s 124ms/step - loss: 0.1841 - accuracy: 0.9406 - val_loss: 0.4423 - val_accuracy: 0.8859
Epoch 4/10
675/675 [==============================] - 82s 122ms/step - loss: 0.1623 - accuracy: 0.9456 - val_loss: 0.4492 - val_accuracy: 0.8930
Epoch 5/10
675/675 [==============================] - 82s 121ms/step - loss: 0.1479 - accuracy: 0.9512 - val_loss: 0.4820 - val_accuracy: 0.8848
Epoch 6/10
675/675 [==============================] - 81s 121ms/step - loss: 0.1545 - accuracy: 0.9516 - val_loss: 0.6084 - val_accuracy: 0.8830
Epoch 7/10
675/675 [==============================] - 86s 127ms/step - loss: 0.1236 - accuracy: 0.9600 - val_loss: 0.6586 - val_ac

In [22]:
epochs = 4


base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

#classification layers
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

#transfer learning model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_ds, epochs=epochs, batch_size=batch_size, validation_data=val_ds)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_ds)
print("Test Accuracy:", test_accuracy)



94765736/94765736 [==============================] - 6s 0us/step
Epoch 1/4



675/675 [==============================] - 66s 95ms/step - loss: 0.4341 - accuracy: 0.8794 - val_loss: 0.2433 - val_accuracy: 0.9170
Epoch 2/4
675/675 [==============================] - 69s 103ms/step - loss: 0.1641 - accuracy: 0.9458 - val_loss: 0.2339 - val_accuracy: 0.9200
Epoch 3/4
675/675 [==============================] - 73s 109ms/step - loss: 0.1076 - accuracy: 0.9640 - val_loss: 0.2555 - val_accuracy: 0.9263
Epoch 4/4
85/85 [==============================] - 8s 91ms/step - loss: 0.3685 - accuracy: 0.9144
Test Accuracy: 0.9144444465637207


In [23]:
epochs = 4


base_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

#classification layers
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

#transfer learning model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_ds, epochs=epochs, batch_size=batch_size, validation_data=val_ds)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_ds)
print("Test Accuracy:", test_accuracy)


258076736/258076736 [==============================] - 19s 0us/step
Epoch 1/4
675/675 [==============================] - 184s 253ms/step - loss: 1.3172 - accuracy: 0.5668 - val_loss: 0.7539 - val_accuracy: 0.7496
Epoch 2/4
675/675 [==============================] - 166s 245ms/step - loss: 0.7128 - accuracy: 0.7610 - val_loss: 0.7173 - val_accuracy: 0.7619
Epoch 3/4
675/675 [==============================] - 172s 254ms/step - loss: 0.6013 - accuracy: 0.7937 - val_loss: 0.6379 - val_accuracy: 0.7885
Epoch 4/4
85/85 [==============================] - 16s 190ms/step - loss: 0.5520 - accuracy: 0.8115
Test Accuracy: 0.8114814758300781


In [29]:
batch_size = 32
img_height = 64
img_width = 64
num_classes = 10
epochs = 4

(train_ds, val_ds, test_ds), metadata = tfds.load(
    'eurosat',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
    batch_size=batch_size
)

class_labels = metadata.features['label'].names


epochs = 4


base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

#classification layers
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

#transfer learning model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
retrainedmodel = model.fit(train_ds, epochs=epochs, batch_size=batch_size, validation_data=val_ds)
model.save("demo4model.h5")

Epoch 1/4
675/675 [==============================] - 64s 92ms/step - loss: 0.4116 - accuracy: 0.8838 - val_loss: 0.2889 - val_accuracy: 0.9004
Epoch 2/4
675/675 [==============================] - 60s 89ms/step - loss: 0.1662 - accuracy: 0.9444 - val_loss: 0.2524 - val_accuracy: 0.9193
Epoch 3/4
675/675 [==============================] - 60s 89ms/step - loss: 0.0996 - accuracy: 0.9658 - val_loss: 0.2473 - val_accuracy: 0.9281
Epoch 4/4
675/675 [==============================] - 60s 89ms/step - loss: 0.0786 - accuracy: 0.9724 - val_loss: 0.3048 - val_accuracy: 0.9252


C:\Users\sasui\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
